<a href="https://colab.research.google.com/github/IKKEM-Lin/colab/blob/main/DOI_Handler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crossrefapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [2]:
from crossref.restful import Works
import json
works = Works()


In [3]:
with open("dois2024.json", "r") as f:
  all_doi = json.load(f)

# with open("result.json", "r") as f:
#   result = json.load(f) or {}

# for ind, doi in enumerate(all_doi):
#   if result.get(doi):
#     continue
#   print(f"start {ind}")
#   temp = works.doi(doi)
#   result[doi] = temp

## 多线程获取doi元数据（耗时较长）

In [ ]:
import concurrent.futures
import requests

result_dict = {}
total = 0

def fetch_doi(doi):
  works = Works()
  return {doi: works.doi(doi)}

def progress_callback(future):
  global total
  # 在这个回调函数中处理任务完成的逻辑，例如显示进度
  total+=1
  print(f"Task completed: {total}")
# 设置线程池的最大线程数
max_workers = 10

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
  # 提交任务并获取结果
  # results = list(executor.map(fetch_doi, all_doi))
  futures = [executor.submit(fetch_doi, url) for url in all_doi]

  # 使用as_completed获取已完成的任务
  for future in concurrent.futures.as_completed(futures):
      progress_callback(future)

Task completed: 1
Task completed: 2
Task completed: 3
Task completed: 4
Task completed: 5
Task completed: 6
Task completed: 7
Task completed: 8
Task completed: 9
Task completed: 10
Task completed: 11
Task completed: 12
Task completed: 13
Task completed: 14
Task completed: 15
Task completed: 16
Task completed: 17
Task completed: 18
Task completed: 19
Task completed: 20
Task completed: 21
Task completed: 22
Task completed: 23
Task completed: 24
Task completed: 25
Task completed: 26
Task completed: 27
Task completed: 28
Task completed: 29
Task completed: 30
Task completed: 31
Task completed: 32
Task completed: 33
Task completed: 34
Task completed: 35
Task completed: 36
Task completed: 37
Task completed: 38
Task completed: 39
Task completed: 40
Task completed: 41
Task completed: 42
Task completed: 43
Task completed: 44
Task completed: 45
Task completed: 46
Task completed: 47
Task completed: 48
Task completed: 49
Task completed: 50
Task completed: 51


In [ ]:
result_dict = {}
for result in futures:
  result_dict.update(result.result())

with open("result2.json", "w") as f:
  json.dump(result_dict, f)

In [ ]:
!curl -k -F "file=@./result2.json" -F "token=jejhiafi2imduwnijbaf" -F "model=2"  -X POST "https://tmp-cli.vx-cdn.com/app/upload_cli"

## 统计各期刊文献数量

In [ ]:
all_pub = []
pub_dois = []
for key, value in result_dict.items():
  pub_dois.append(key)
  container = value.get("container-title", [""])
  all_pub.append(container[0])

In [ ]:
import collections
counter = collections.Counter(all_pub)
print(counter)

Counter({'Journal of Catalysis': 3905, 'ACS Catalysis': 3322, 'Science Advances': 3030, 'Science': 809, 'Science Signaling': 257, 'Science Translational Medicine': 157, 'Science Immunology': 30, 'Green Chemistry': 15, 'Science Robotics': 12, 'Angewandte Chemie International Edition': 5, 'ChemSusChem': 5, 'ACS Sustainable Chemistry &amp; Engineering': 4, 'Journal of the American Chemical Society': 4, 'Industrial &amp; Engineering Chemistry Research': 3, 'RSC Advances': 3, 'Journal of Materials Chemistry A': 3, 'Catalysis Letters': 2, 'Catalysis Today': 2, 'ACS Omega': 2, 'Advanced Synthesis &amp; Catalysis': 1, 'Angewandte Chemie': 1, 'Chemistry – An Asian Journal': 1, 'Chemistry – A European Journal': 1, 'Bioprocess and Biosystems Engineering': 1, 'Industrial Crops and Products': 1, 'Nature Nanotechnology': 1, 'Green Chem.': 1, 'RSC Adv.': 1, 'Catalysis Science &amp; Technology': 1})


In [ ]:
s={}
for key, value in counter.items():
  for key2, value2 in result_dict.items():
    if value2.get("container-title", [""])[0] == key:
      s[key] = f"{value}@@{key2}"
      break

In [ ]:
s

{'ACS Catalysis': '3322@@10.1021/acscatal.0c00007',
 'Advanced Synthesis &amp; Catalysis': '1@@10.1002/adsc.202001003',
 'Angewandte Chemie': '1@@10.1002/ange.201601208',
 'Angewandte Chemie International Edition': '5@@10.1002/anie.201402904',
 'Chemistry – An Asian Journal': '1@@10.1002/asia.201800738',
 'Chemistry – A European Journal': '1@@10.1002/chem.201301735',
 'ChemSusChem': '5@@10.1002/cssc.200900137',
 'Journal of Catalysis': '3905@@10.1006/jcat.1995.1005',
 'Bioprocess and Biosystems Engineering': '1@@10.1007/s00449-019-02205-7',
 'Catalysis Letters': '2@@10.1007/s10562-011-0707-y',
 'Industrial Crops and Products': '1@@10.1016/S0926-6690(98)00037-5',
 'Catalysis Today': '2@@10.1016/j.cattod.2010.06.004',
 'Industrial &amp; Engineering Chemistry Research': '3@@10.1021/acs.iecr.0c00937',
 'ACS Omega': '2@@10.1021/acsomega.0c02178',
 'ACS Sustainable Chemistry &amp; Engineering': '4@@10.1021/acssuschemeng.0c01319',
 'Journal of the American Chemical Society': '4@@10.1021/ja058